In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from utils import *
import torch
import torch.nn as nn


In [2]:
# superclass of modules
class Module:
    """
    Module is a super class. It could be a single layer, or a multilayer perceptron.
    """
    
    def __init__(self):
        self.train = True
        return
    
    def forward(self, _input):
        """
        h = f(z); z is the input, and h is the output.
        
        Inputs:
        _input: z
        
        Returns:
        output h
        """
        pass
    
    def backward(self, _input, _gradOutput):
        """
        Compute:
        gradient w.r.t. _input
        gradient w.r.t. trainable parameters
        
        Inputs:
        _input: z
        _gradOutput: dL/dh
        
        Returns:
        gradInput: dL/dz
        """
        pass
        
    def parameters(self):
        """
        Return the value of trainable parameters and its corresponding gradient (Used for grandient descent)
        
        Returns:
        params, gradParams
        """
        pass
    
    def training(self):
        """
        Turn the module into training mode.(Only useful for Dropout layer)
        Ignore it if you are not using Dropout.
        """
        self.train = True
        
    def evaluate(self):
        """
        Turn the module into evaluate mode.(Only useful for Dropout layer)
        Ignore it if you are not using Dropout.
        """
        self.train = False

In [3]:
class Sequential(Module):
    """
    Sequential provides a way to plug layers together in a feed-forward manner.
    """
    def __init__(self):
        Module.__init__(self)
        self.layers = [] # layers contain all the layers in order
    
    def add(self, layer):
        self.layers.append(layer) # Add another layer at the end
    
    def size(self):
        return len(self.layers) # How many layers.
    
    def forward(self, _input):
        """
        Feed forward through all the layers, and return the output of the last layer
        """
        # self._inputs saves the input of each layer
        # self._inputs[i] is the input of i-th layer
        self._inputs = [_input]
        for i in range(self.size()):
            self._inputs.append(self.layers[i].forward(self._inputs[i]))
        self._output = self._inputs[-1]
        return self._output
    
    def backward(self, _input, _gradOutput):
        """
        Backpropogate through all the layers using chain rule.
        """
        # self._gradInputs[i] is the gradient of loss w.r.t. the input of i-th layer
        self._gradInputs = [None] * (self.size() + 1)  
        self._gradInputs[self.size()] = _gradOutput
        for i in reversed(range(self.size())):
            self._gradInputs[i] = self.layers[i].backward(self._inputs[i], self._gradInputs[i + 1])
        self._gradInput = self._gradInputs[0]
        return self._gradInput
    
    def parameters(self):
        """
        Return trainable parameters and its corresponding gradient in a nested list
        """
        params = []
        gradParams = []
        for m in self.layers:
            _p, _g = m.parameters()
            if _p is not None:
                params.append(_p)
                gradParams.append(_g)
        return params, gradParams

    def training(self):
        """
        Turn all the layers into training mode
        """
        Module.training(self)
        for m in self.layers:
            m.training()
    
    def evaluate(self):
        """
        Turn all the layers into evaluate mode
        """
        Module.evaluate(self)
        for m in self.layers:
            m.evaluate()

In [4]:
class FullyConnected(Module):
    """
    Fully connected layer
    """
    def __init__(self, inputSize, outputSize):
        Module.__init__(self)
        # Initalization
        stdv = 1./np.sqrt(inputSize)
        
        self.weight = np.random.uniform(-stdv, stdv, (inputSize, outputSize))
        self.gradWeight = np.zeros((inputSize, outputSize))
        self.bias = np.random.uniform(-stdv, stdv, outputSize)
        self.gradBias = np.zeros(outputSize)
        
    def forward(self, _input):
        """
        output = W * input + b
        
        _input:
        N x inputSize matrix
        
        """
        _input = _input.reshape(_input.shape[0], -1)

        self._output = np.dot(_input, self.weight) + self.bias
        return self._output
    
    def backward(self, _input, _gradOutput):
        """
        _input:
        N x inputSize matrix
        _gradOutputSize:
        N x outputSize matrix
        """
        self.gradWeight = np.dot(_input.transpose(),_gradOutput)
        self.gradBias = np.sum(_gradOutput,0)
        self._gradInput = np.dot(_gradOutput,self.weight.transpose())
        return self._gradInput
        
    def parameters(self):
        """
        Return weight and bias and their g
        """
        return [self.weight, self.bias], [self.gradWeight, self.gradBias]

In [5]:
class ReLU(Module):
    """
    ReLU activation, not trainable.
    """
    def __init__(self):
        Module.__init__(self)
        return
    
    def forward(self, _input):
        """
        output = max(0, input)
        
        _input:
        N x d matrix
        """
        self._output = np.maximum(0, _input)
        return self._output
    
    def backward(self, _input, _gradOutput):
        """
        gradInput = gradOutput * mask
        mask = _input > 0
        
        _input:
        N x d matrix
        
        _gradOutput:
        N x d matrix
        """
        mask = _input > 0
        self._gradInput = _gradOutput * mask
        return self._gradInput

        
    def parameters(self):
        """
        No trainable parametersm, return None
        """
        return None, None

In [6]:
# Optional
class Logistic(Module):
    """
    Logistic activation, not trainable.
    """
    def __init__(self):
        Module.__init__(self)
        return
    
    def forward(self, _input):
        self._input = _input
        self._output = 1. /  (1 + np.exp(-self._input))
        return self._output
    
    def backward(self, _input, _gradOutput):
        self._gradInput = _gradOutput * (1. - self._output) * self._output
        return self._gradInput
        
    def parameters(self):
        return None, None

In [7]:
# Optional
class Dropout(Module):
    """
    A dropout layer
    """
    def __init__(self, p = 0.5):
        Module.__init__(self)
        self.p = p #self.p is the drop rate, if self.p is 0, then it's a identity layer
        
    def forward(self, _input):
        self._output = _input
        # YOUR CODE HERE
        # Need to take care of training mode and evaluation mode
        if self.p > 0:
            if self.train:
                self.mask = np.random.binomial(1, 1 - self.p, _input.shape).astype('float64')
                self.mask /= 1 - self.p
                self._output *= self.mask
        return self._output
    
    def backward(self, _input, _gradOutput):
        self._gradInput = _gradOutput
        if self.train:
            if self.p > 0:
                self._gradInput *= self.mask
        return self._gradInput
    
    def parameters(self):
        """
        No trainable parameters.
        """
        return None, None

In [8]:
class SoftMaxLoss(object):
    def __init__(self):
        return
        
    def forward(self, _input, _label):
        """
        Softmax and cross entropy loss layer. Should return a scalar, since it's a
        loss. (It's almost identical to what in hw2)

        _input: N x C
        _labels: N x C, one-hot

        Returns: loss (scalar)
        """
        self._input = _input - np.max(_input, axis=1, keepdims=True)
        self._logprob = self._input - np.log(np.exp(self._input).sum(axis=1, keepdims=True))
        self._output = np.mean(np.sum(-self._logprob * _label, axis=1))
        return self._output
    
    def backward(self, _input, _label):
        self._gradInput = (np.exp(self._logprob) - _label) / _label.shape[0]
        return self._gradInput

In [9]:
# Test softmaxloss, the relative error should be small enough
def test_sm():
    crit = SoftMaxLoss()
    gt = np.zeros((3, 10))
    gt[np.arange(3), np.array([1,2,3])] = 1
    x = np.random.random((3,10))
    def test_f(x):
        return crit.forward(x, gt)

    crit.forward(x, gt)

    gradInput = crit.backward(x, gt)
    gradInput_num = numeric_gradient(test_f, x, 1, 1e-6)
    print(gradInput)
    print(gradInput_num)
    print(relative_error(gradInput, gradInput_num, 1e-8))
    
test_sm()

[[ 0.03419546 -0.30100657  0.02007918  0.04038765  0.02561507  0.03003862
   0.02748343  0.04419535  0.03635619  0.04265563]
 [ 0.04053102  0.04722263 -0.31209795  0.05100902  0.02184379  0.03082003
   0.02457761  0.0324217   0.02202906  0.04164309]
 [ 0.03689008  0.02017514  0.04039379 -0.30474866  0.03439409  0.05277443
   0.02983538  0.03036936  0.03903806  0.02087834]]
[[ 0.03419546 -0.30100657  0.02007918  0.04038765  0.02561507  0.03003862
   0.02748343  0.04419535  0.03635619  0.04265563]
 [ 0.04053102  0.04722263 -0.31209795  0.05100902  0.02184379  0.03082003
   0.02457761  0.0324217   0.02202906  0.04164309]
 [ 0.03689008  0.02017514  0.04039379 -0.30474866  0.03439409  0.05277443
   0.02983538  0.03036936  0.03903806  0.02087834]]
5.6386797378842845e-09


In [10]:
# Test modules, all the relative errors should be small enough
def test_module(model):

    #model.evaluate()

    crit = TestCriterion()
    gt = np.random.random((3,10))
    x = np.random.random((3,10))
    def test_f(x):
        return crit.forward(model.forward(x), gt)

    gradInput = model.backward(x, crit.backward(model.forward(x), gt))
    gradInput_num = numeric_gradient(test_f, x, 1, 1e-6)
    print(relative_error(gradInput, gradInput_num, 1e-8))

# Test fully connected
model = FullyConnected(10, 10)
print('testing FullyConnected')
test_module(model)

# Test ReLU
print('testing ReLU')
model = ReLU()
test_module(model)

# Test Dropout
print("testing dropout")
model = Dropout()
test_module(model)
# You can only test dropout in evaluation mode.

# Test Sequential
model = Sequential()
model.add(FullyConnected(10, 10))
model.add(ReLU())
model.add(FullyConnected(10, 10))
#model.add(Dropout())
test_module(model)

testing FullyConnected
1.1215437853340808e-08
testing ReLU
6.988901124005461e-11
testing dropout
1.0
1.1236146222602319e-08


In [11]:
# Test gradient descent, the loss should be lower and lower
trainX = np.random.random((10,5))

model = Sequential()
model.add(FullyConnected(5, 3))
model.add(ReLU())
#model.add(Dropout())
model.add(FullyConnected(3, 1))

crit = TestCriterion()

it = 0
state = None
while True:
    output = model.forward(trainX)
    loss = crit.forward(output, None)
    if it % 100 == 0:
        print(loss)
    doutput = crit.backward(output, None)
    model.backward(trainX, doutput)
    params, gradParams = model.parameters()
    sgdmom(params, gradParams, 0.01, 0.8)
    if it > 1000:
        break
    it += 1

0.3943679846996052
0.013860989469357432
0.01092164763738045
0.013228630714962119
0.012597156017384762
0.012148466808700612
0.01204733344916826
0.011837429413871229
0.013465227329659346
0.010133790280275251
0.014605215658158779


Now we start to work on real data.

In [12]:
import MNIST_utils
data_fn = "CLEAN_MNIST_SUBSETS.h5"

# We only consider large set this time
print("Load large trainset.")
Xlarge,Ylarge = MNIST_utils.load_data(data_fn, "large_train")
print(Xlarge.shape)
print(Ylarge.shape)

print("Load valset.")
Xval,Yval = MNIST_utils.load_data(data_fn, "val")
print(Xval.shape)
print(Yval.shape)

Load large trainset.
(7000, 576)
(7000, 10)
Load valset.
(2000, 576)
(2000, 10)


In [13]:
def predict(X, model):
    """
    Evaluate the soft predictions of the model.
    Input:
    X : N x d array (no unit terms)
    model : a multi-layer perceptron
    Output:
    yhat : N x C array
        yhat[n][:] contains the score over C classes for X[n][:]
    """
    return model.forward(X)

def error_rate(X, Y, model):
    """
    Compute error rate (between 0 and 1) for the model
    """
    model.evaluate()
    res = 1 - (model.forward(X).argmax(-1) == Y.argmax(-1)).mean()
    model.training()
    return res

from copy import deepcopy

def runTrainVal(X,Y,model,Xval,Yval,trainopt):
    """
    Run the train + evaluation on a given train/val partition
    trainopt: various (hyper)parameters of the training procedure
    During training, choose the model with the lowest validation error. (early stopping)
    """
    
    eta = trainopt['eta']
    
    N = X.shape[0] # number of data points in X
    
    # Save the model with lowest validation error
    minValError = np.inf
    saved_model = None
    
    shuffled_idx = np.random.permutation(N)
    start_idx = 0
    for iteration in range(trainopt['maxiter']):
        if iteration % int(trainopt['eta_frac'] * trainopt['maxiter']) == 0:
            eta *= trainopt['etadrop']
        # form the next mini-batch
        stop_idx = min(start_idx + trainopt['batch_size'], N)
        batch_idx = range(N)[int(start_idx):int(stop_idx)]
        bX = X[shuffled_idx[batch_idx],:]
        bY = Y[shuffled_idx[batch_idx],:]

        score = model.forward(bX)
        loss = crit.forward(score, bY)
        # print(loss)
        dscore = crit.backward(score, bY)
        model.backward(bX, dscore)
        
        # Update the data using 
        params, gradParams = model.parameters()
        sgdmom(params, gradParams, eta, weight_decay = trainopt['lambda'])    
        start_idx = stop_idx % N
        
        if (iteration % trainopt['display_iter']) == 0:
            #compute train and val error; multiply by 100 for readability (make it percentage points)
            trainError = 100 * error_rate(X, Y, model)
            valError = 100 * error_rate(Xval, Yval, model)
            print('{:8} batch loss: {:.3f} train error: {:.3f} val error: {:.3f}'.format(iteration, loss, trainError, valError))
            
            if valError < minValError:
                saved_model = deepcopy(model)
                minValError = valError
        
    return saved_model, minValError, trainError

In [14]:
def build_model(input_size, hidden_size, output_size, activation_func = 'ReLU', dropout = 0):
    """
    Build the model:
    input_size: the dimension of input data
    hidden_size: the dimension of hidden vector
    output_size: the output size of final layer.
    activation_func: ReLU, Logistic, Tanh, etc. (Need to be implemented by yourself)
    dropout: the dropout rate: if dropout == 0, this is equivalent to no dropout
    """
    model = Sequential()
    model.add(FullyConnected(input_size, hidden_size))

    if activation_func == 'ReLU':
        model.add(ReLU())
    elif activation_func == 'Logistic':
        model.add(Logistic())

    if dropout > 0:
        model.add(Dropout(dropout))

    model.add(FullyConnected(hidden_size, output_size))

    return model

In [16]:
# -- training options
trainopt = {
    'eta': .1,   # initial learning rate
    'maxiter': 20000,   # max number of iterations (updates) of SGD
    'display_iter': 500,  # display batch loss every display_iter updates
    'batch_size': 100,  
    'etadrop': .5, # when dropping eta, multiply it by this number (e.g., .5 means halve it)
    'eta_frac': .25  #
}

NFEATURES = Xlarge.shape[1]

# we will maintain a record of models trained for different values of lambda
# these will be indexed directly by lambda value itself
trained_models = dict()

# set the (initial?) set of lambda values to explore
lambdas = np.array([0.01, 0.025, 0.05, 0.075, 0.1])
hidden_sizes = np.array([10])
    
for lambda_ in lambdas:
    for hidden_size_ in hidden_sizes:
        trainopt['lambda'] = lambda_
        model = build_model(NFEATURES, hidden_size_, 10, dropout = 0)
        crit = SoftMaxLoss()
        # -- model trained on large train set
        trained_model,valErr,trainErr = runTrainVal(Xlarge, Ylarge, model, Xval, Yval, trainopt)
        trained_models[(lambda_, hidden_size_)] = {'model': trained_model, "val_err": valErr, "train_err": trainErr }
        print('train set model: -> lambda= %.4f, train error: %.2f, val error: %.2f' % (lambda_, trainErr, valErr))
    
best_trained_lambda = 0.
best_trained_model = None
best_trained_val_err = 100.
for (lambda_, hidden_size_), results in trained_models.items():
    print('lambda= %.4f, hidden size: %5d, val error: %.2f' %(lambda_, hidden_size_, results['val_err']))
    if results['val_err'] < best_trained_val_err:
        best_trained_val_err = results['val_err']
        best_trained_model = results['model']
        best_trained_lambda = lambda_

print("Best train model val err:", best_trained_val_err)
print("Best train model lambda:", best_trained_lambda)

       0 batch loss: 2.308 train error: 90.529 val error: 90.600
     500 batch loss: 0.440 train error: 10.671 val error: 10.000
    1000 batch loss: 0.317 train error: 8.571 val error: 8.300
    1500 batch loss: 0.250 train error: 7.457 val error: 7.450
    2000 batch loss: 0.239 train error: 6.629 val error: 7.300
    2500 batch loss: 0.156 train error: 6.186 val error: 7.200
    3000 batch loss: 0.305 train error: 5.857 val error: 7.250
    3500 batch loss: 0.170 train error: 5.486 val error: 7.100
    4000 batch loss: 0.160 train error: 5.186 val error: 7.050
    4500 batch loss: 0.161 train error: 4.657 val error: 6.900
    5000 batch loss: 0.143 train error: 4.371 val error: 7.150
    5500 batch loss: 0.176 train error: 4.243 val error: 7.150
    6000 batch loss: 0.080 train error: 4.014 val error: 6.950
    6500 batch loss: 0.242 train error: 3.943 val error: 7.150
    7000 batch loss: 0.100 train error: 3.857 val error: 7.400
    7500 batch loss: 0.108 train error: 3.757 val e

    3500 batch loss: 0.125 train error: 5.400 val error: 7.150
    4000 batch loss: 0.124 train error: 5.043 val error: 7.200
    4500 batch loss: 0.227 train error: 4.986 val error: 7.050
    5000 batch loss: 0.224 train error: 4.586 val error: 7.200
    5500 batch loss: 0.175 train error: 4.500 val error: 6.750
    6000 batch loss: 0.137 train error: 4.186 val error: 7.000
    6500 batch loss: 0.109 train error: 4.043 val error: 6.950
    7000 batch loss: 0.087 train error: 3.886 val error: 6.900
    7500 batch loss: 0.089 train error: 3.743 val error: 7.000
    8000 batch loss: 0.171 train error: 3.671 val error: 6.750
    8500 batch loss: 0.187 train error: 3.514 val error: 6.900
    9000 batch loss: 0.145 train error: 3.557 val error: 6.800
    9500 batch loss: 0.117 train error: 3.429 val error: 6.850
   10000 batch loss: 0.094 train error: 3.214 val error: 6.950
   10500 batch loss: 0.072 train error: 3.071 val error: 6.900
   11000 batch loss: 0.073 train error: 3.100 val error

In [ ]:
#Generate a Kaggle submission file using `model`
kaggleX = MNIST_utils.load_data(data_fn, 'kaggle')
kaggleYhat = predict(kaggleX, best_trained_model).argmax(-1)
save_submission('submission-mnist.csv', kaggleYhat)

For SGD, the performance was:

- best train set model: -> lambda= 0.1000, train error: 2.30, val error: 6.85
- lambda= 0.0000, hidden size: 10, val error: 7.10
- lambda= 0.0010, hidden size: 10, val error: 7.05
- lambda= 0.0100, hidden size: 10, val error: 7.30
- lambda= 0.1000, hidden size: 10, val error: 6.85

For SGD with momentum:

- best train set model: -> lambda= 0.1000, train error: 2.51, val error: 6.70
- lambda= 0.0000, hidden size: 10, val error: 7.30
- lambda= 0.0010, hidden size: 10, val error: 6.80
- lambda= 0.0100, hidden size: 10, val error: 7.60
- lambda= 0.1000, hidden size: 10, val error: 6.70

For SGD with Nesterov momentum:

- train set model: -> lambda= 0.0200, train error: 2.49, val error: 6.60
- lambda= 0.0000, hidden size:    10, val error: 7.10
- lambda= 0.0500, hidden size:    10, val error: 7.15
- lambda= 0.0100, hidden size:    10, val error: 7.30
- lambda= 0.0150, hidden size:    10, val error: 7.40
- lambda= 0.0200, hidden size:    10, val error: 6.60

Overall, SGD with Nesterov momentum outperforms the other two algorithms in terms of achieving the lowest validation error. This suggests that incorporating Nesterov momentum into the optimization process helps in finding a better solution for the linear regression problem.
